# Best Hybrid Search Configuration

This notebook runs different hybrid search configurations, calculates the metrics for each configuration and compares the results to the metrics calcuated after the baseline run from the previous notebook. 

We are using the same query set to have a fair comparison.

## Get queries

In [137]:
import pandas as pd
import requests
import pytrec_eval
import json
import mercury as mr
import itertools
from IPython.display import display, HTML, Image

app = mr.App(title="Let's Run a Hybrid Search", static_notebook=True)

mercury.App

In [138]:
name = '../queries.txt'
df_query_idx = pd.read_csv(name, sep="\t", names=['idx', 'query'])

In [139]:
df_query_idx

,idx,query
0,0,$30 roblox gift card not digital
1,1,(fiction without frontiers)
2,2,100
3,3,10x10x6 cake box without window
4,4,15 inch light weight laptop that has lots of m...
...,...,...
215,215,wooden stool
216,216,woodwick wax melt
217,217,world of warcraft anniversary collector's edition
218,218,wowled


In [140]:
name = '../ratings.qrels'

df_ratings = pd.read_csv(name, sep="\t", names=['idx', 'docid', 'rating'])#, index=False)
df_ratings

,idx,docid,rating
0,0,B07RX6FBFR,3
1,0,B09194H44R,0
2,0,B08R5N6W6B,2
3,0,B07Y693ND1,0
4,0,B07RZ75JW3,2
...,...,...,...
4060,219,B00JX10Q2O,2
4061,219,B00KY41UHO,3
4062,219,B00QXJOUL2,3
4063,219,B00UY14WCM,3


## Query OpenSearch with the Hybrid Search Configurations

In [141]:
keyword_weight = 0.3

In [142]:
neural_weight = round(1.0 - keyword_weight, 2)
print(f"Keyword Weight is {keyword_weight} and Neural Weight is {neural_weight}")

Keyword Weight is 0.3 and Neural Weight is 0.7


In [143]:
# Get model_id
# We are assuming that the installation has only one model. Change this if you have more models 
# and need to pick a specific one

url = "http://localhost:9200/_plugins/_ml/models/_search"

headers = {
    'Content-Type': 'application/json'
}

payload = {
  "query": {
    "match_all": {}
  },
  "size": 1
}

response = requests.request("POST", url, headers=headers, data=json.dumps(payload))

model_id = response.json()['hits']['hits'][0]['_source']['model_id']

In [144]:
normalization = 'arithmetic_mean'
combination = 'l2'
keyword = 0.3
vector = 0.7
pipeline = 'hybrid-search-pipeline'

url = "http://localhost:9200/_search/pipeline/" + pipeline

print(f"Setting default model id to: {model_id}")
payload = {
  "request_processors": [
    {
      "neural_query_enricher" : {
        "description": "Sets the default model ID at index and field levels",
        "default_model_id": model_id,
        "neural_field_default_id": {
           "title_embeddings": model_id
        }
      }
    }
  ],
  "phase_results_processors": [
    {
      "normalization-processor": {
        "normalization": {
          "technique": "min_max"
        },
        "combination": {
          "technique": "arithmetic_mean",
          "parameters": {
            "weights": [
              keyword_weight,
              neural_weight
            ]
          }
        }
      }
    }
  ]    
}


response = requests.request("PUT", url, headers=headers, data=json.dumps(payload))
mr.JSON(response.json(), level=4)

Setting default model id to: 1Zr9VpIBFSlgWAuGFzOL


In [145]:
df_query_idx

,idx,query
0,0,$30 roblox gift card not digital
1,1,(fiction without frontiers)
2,2,100
3,3,10x10x6 cake box without window
4,4,15 inch light weight laptop that has lots of m...
...,...,...
215,215,wooden stool
216,216,woodwick wax melt
217,217,world of warcraft anniversary collector's edition
218,218,wowled


## Create a DataFrame with all possible combinations of hybrid search combination

In [146]:
# Define the possible values for each column
normalization_values = ['min_max', 'l2']
combination_values = ['arithmetic_mean', 'harmonic_mean', 'geometric_mean']
keyword_values = [round(i * 0.1, 1) for i in range(11)]

# Create all possible combinations of normalization, combination, and keyword
combinations = list(itertools.product(normalization_values, combination_values, keyword_values))

# Calculate the vector as 1.0 - keyword
data = [(norm, comb, kw, 1.0 - kw) for norm, comb, kw in combinations]

# Create DataFrame
df_hybrid_search_params = pd.DataFrame(data, columns=['normalization', 'combination', 'keyword', 'vector'])

# Create a column with a pipeline name made up of its components
df_hybrid_search_params['pipeline'] = df_hybrid_search_params.normalization.apply(str) + \
    df_hybrid_search_params.combination.apply(str) + df_hybrid_search_params.keyword.apply(str)

df_hybrid_search_params.head()

,normalization,combination,keyword,vector,pipeline
0,min_max,arithmetic_mean,0.0,1.0,min_maxarithmetic_mean0.0
1,min_max,arithmetic_mean,0.1,0.9,min_maxarithmetic_mean0.1
2,min_max,arithmetic_mean,0.2,0.8,min_maxarithmetic_mean0.2
3,min_max,arithmetic_mean,0.3,0.7,min_maxarithmetic_mean0.3
4,min_max,arithmetic_mean,0.4,0.6,min_maxarithmetic_mean0.4


In [147]:
def create_search_pipeline(df):
    for idx, row in df.iterrows():
        normalization = row['normalization']
        combination = row['combination']
        keyword = row['keyword']
        vector = round(row['vector'],1)
        pipeline = row['pipeline']

        payload = {
          "request_processors": [
            {
              "neural_query_enricher" : {
                "description": "Sets the default model ID at index and field levels",
                "default_model_id": model_id,
                "neural_field_default_id": {
                   "title_embeddings": model_id
                }
              }
            }
          ],
          "phase_results_processors": [
            {
              "normalization-processor": {
                "normalization": {
                  "technique": normalization
                },
                "combination": {
                  "technique": combination,
                  "parameters": {
                    "weights": [
                      keyword,
                      vector
                    ]
                  }
                }
              }
            }
          ]    
        }

        url = "http://localhost:9200/_search/pipeline/" + pipeline
        
        response = requests.request("PUT", url, headers=headers, data=json.dumps(payload))
        mr.JSON(response.json(), level=1)
        print(payload)

In [148]:
create_search_pipeline(df_hybrid_search_params)

{'request_processors': [{'neural_query_enricher': {'description': 'Sets the default model ID at index and field levels', 'default_model_id': '1Zr9VpIBFSlgWAuGFzOL', 'neural_field_default_id': {'title_embeddings': '1Zr9VpIBFSlgWAuGFzOL'}}}], 'phase_results_processors': [{'normalization-processor': {'normalization': {'technique': 'min_max'}, 'combination': {'technique': 'arithmetic_mean', 'parameters': {'weights': [0.0, 1.0]}}}}]}


{'request_processors': [{'neural_query_enricher': {'description': 'Sets the default model ID at index and field levels', 'default_model_id': '1Zr9VpIBFSlgWAuGFzOL', 'neural_field_default_id': {'title_embeddings': '1Zr9VpIBFSlgWAuGFzOL'}}}], 'phase_results_processors': [{'normalization-processor': {'normalization': {'technique': 'min_max'}, 'combination': {'technique': 'arithmetic_mean', 'parameters': {'weights': [0.1, 0.9]}}}}]}


{'request_processors': [{'neural_query_enricher': {'description': 'Sets the default model ID at index and field levels', 'default_model_id': '1Zr9VpIBFSlgWAuGFzOL', 'neural_field_default_id': {'title_embeddings': '1Zr9VpIBFSlgWAuGFzOL'}}}], 'phase_results_processors': [{'normalization-processor': {'normalization': {'technique': 'min_max'}, 'combination': {'technique': 'arithmetic_mean', 'parameters': {'weights': [0.2, 0.8]}}}}]}


{'request_processors': [{'neural_query_enricher': {'description': 'Sets the default model ID at index and field levels', 'default_model_id': '1Zr9VpIBFSlgWAuGFzOL', 'neural_field_default_id': {'title_embeddings': '1Zr9VpIBFSlgWAuGFzOL'}}}], 'phase_results_processors': [{'normalization-processor': {'normalization': {'technique': 'min_max'}, 'combination': {'technique': 'arithmetic_mean', 'parameters': {'weights': [0.3, 0.7]}}}}]}


{'request_processors': [{'neural_query_enricher': {'description': 'Sets the default model ID at index and field levels', 'default_model_id': '1Zr9VpIBFSlgWAuGFzOL', 'neural_field_default_id': {'title_embeddings': '1Zr9VpIBFSlgWAuGFzOL'}}}], 'phase_results_processors': [{'normalization-processor': {'normalization': {'technique': 'min_max'}, 'combination': {'technique': 'arithmetic_mean', 'parameters': {'weights': [0.4, 0.6]}}}}]}


...

{'request_processors': [{'neural_query_enricher': {'description': 'Sets the default model ID at index and field levels', 'default_model_id': '1Zr9VpIBFSlgWAuGFzOL', 'neural_field_default_id': {'title_embeddings': '1Zr9VpIBFSlgWAuGFzOL'}}}], 'phase_results_processors': [{'normalization-processor': {'normalization': {'technique': 'l2'}, 'combination': {'technique': 'geometric_mean', 'parameters': {'weights': [0.6, 0.4]}}}}]}


{'request_processors': [{'neural_query_enricher': {'description': 'Sets the default model ID at index and field levels', 'default_model_id': '1Zr9VpIBFSlgWAuGFzOL', 'neural_field_default_id': {'title_embeddings': '1Zr9VpIBFSlgWAuGFzOL'}}}], 'phase_results_processors': [{'normalization-processor': {'normalization': {'technique': 'l2'}, 'combination': {'technique': 'geometric_mean', 'parameters': {'weights': [0.7, 0.3]}}}}]}


{'request_processors': [{'neural_query_enricher': {'description': 'Sets the default model ID at index and field levels', 'default_model_id': '1Zr9VpIBFSlgWAuGFzOL', 'neural_field_default_id': {'title_embeddings': '1Zr9VpIBFSlgWAuGFzOL'}}}], 'phase_results_processors': [{'normalization-processor': {'normalization': {'technique': 'l2'}, 'combination': {'technique': 'geometric_mean', 'parameters': {'weights': [0.8, 0.2]}}}}]}


{'request_processors': [{'neural_query_enricher': {'description': 'Sets the default model ID at index and field levels', 'default_model_id': '1Zr9VpIBFSlgWAuGFzOL', 'neural_field_default_id': {'title_embeddings': '1Zr9VpIBFSlgWAuGFzOL'}}}], 'phase_results_processors': [{'normalization-processor': {'normalization': {'technique': 'l2'}, 'combination': {'technique': 'geometric_mean', 'parameters': {'weights': [0.9, 0.1]}}}}]}


{'request_processors': [{'neural_query_enricher': {'description': 'Sets the default model ID at index and field levels', 'default_model_id': '1Zr9VpIBFSlgWAuGFzOL', 'neural_field_default_id': {'title_embeddings': '1Zr9VpIBFSlgWAuGFzOL'}}}], 'phase_results_processors': [{'normalization-processor': {'normalization': {'technique': 'l2'}, 'combination': {'technique': 'geometric_mean', 'parameters': {'weights': [1.0, 0.0]}}}}]}


In [149]:
df_relevance = pd.DataFrame()
for config in df_hybrid_search_params.itertuples():
    pipeline_name = config[5]
    print(pipeline_name)

    # Set pipeline 
     
    url = "http://localhost:9200/ecommerce/_search?search_pipeline=" + pipeline_name
    
    headers = {
        'Content-Type': 'application/json'
    }
    # iterate over all query strings and send a hybrid search query to OpenSearch with the set pipeline
    for query in df_query_idx.itertuples():
    
        payload = {
          "_source": {
            "excludes": [
              "title_embedding"
            ]
          },
          "query": {
            "hybrid": {
              "queries": [
                {
                  "multi_match" : {
                      "type":       "best_fields",
                      "fields":     [
                        "product_id^100",
                        "product_bullet_point^3",
                        "product_color^2",
                        "product_brand^5",
                        "product_description",
                        "product_title^10"
                      ],
                      "operator":   "and",
                      "query":      query[2]
                    }
                },
                {
                  "neural": {
                    "title_embedding": {
                      "query_text": query[2],
                      "k": 50
                    }
                  }
                }
              ]
            }
          }
        }
    
        response = requests.request("POST", url, headers=headers, data=json.dumps(payload)).json()
        # store results per pipeline_id
        position = 0
        for hit in response['hits']['hits']:
            # create a new row for the DataFrame and append it
            row = { 'query_id' : str(query[1]), 'query_string': query[2], 'product_id' : hit["_id"], 'position' : str(position), 'relevance' : hit["_score"], 'run': pipeline_name }
    
            new_row_df = pd.DataFrame([row])
            df_relevance = pd.concat([df_relevance, new_row_df], ignore_index=True)
            #print("%(id)s %(title)s: %(name)s" % hit["_source"])
            position += 1
    
    # work with two for loops:
    # 1) one to iterate over the list of queries and have a query id instead of a query
    # 2) another one to iterate over the result sets to have the position of the result in the result set 
    
    # DataFrame with columns:
    # query_id: the id of the query as the trec_eval tool needs a numeric id rather than a query string as an identifier
    # product_id: the id of the product in the hit list
    # position: the position of the product in the result set
    # relevance: relevance as given by the search engine
    # run: the name of the query pipeline

min_maxarithmetic_mean0.0
min_maxarithmetic_mean0.1
min_maxarithmetic_mean0.2
min_maxarithmetic_mean0.3
min_maxarithmetic_mean0.4
min_maxarithmetic_mean0.5
min_maxarithmetic_mean0.6
min_maxarithmetic_mean0.7
min_maxarithmetic_mean0.8
min_maxarithmetic_mean0.9
min_maxarithmetic_mean1.0
min_maxharmonic_mean0.0
min_maxharmonic_mean0.1
min_maxharmonic_mean0.2
min_maxharmonic_mean0.3
min_maxharmonic_mean0.4
min_maxharmonic_mean0.5
min_maxharmonic_mean0.6
min_maxharmonic_mean0.7
min_maxharmonic_mean0.8
min_maxharmonic_mean0.9
min_maxharmonic_mean1.0
min_maxgeometric_mean0.0
min_maxgeometric_mean0.1
min_maxgeometric_mean0.2
min_maxgeometric_mean0.3
min_maxgeometric_mean0.4
min_maxgeometric_mean0.5
min_maxgeometric_mean0.6
min_maxgeometric_mean0.7
min_maxgeometric_mean0.8
min_maxgeometric_mean0.9
min_maxgeometric_mean1.0
l2arithmetic_mean0.0
l2arithmetic_mean0.1
l2arithmetic_mean0.2
l2arithmetic_mean0.3
l2arithmetic_mean0.4
l2arithmetic_mean0.5
l2arithmetic_mean0.6
l2arithmetic_mean0.7
l2arith

In [150]:
df_relevance.head(3)

,query_id,query_string,product_id,position,relevance,run
0,0,$30 roblox gift card not digital,B00F4CF4PU,0,1.000000,min_maxarithmetic_mean0.0
1,0,$30 roblox gift card not digital,B07C438TMN,1,0.474647,min_maxarithmetic_mean0.0
2,0,$30 roblox gift card not digital,B00XJZHJCA,2,0.462698,min_maxarithmetic_mean0.0


# Calculate metrics per pipeline

## Transform data to meet the `pytrec_eval` requirements

### Convert string ids to integer values

In [151]:
unique_ids = pd.Series(pd.concat([df_relevance['product_id'], df_ratings['docid']]).unique())

# Create a mapping of each unique identifier to an integer
id_to_int = {id_val: idx for idx, id_val in enumerate(unique_ids, start=1)}

# Map the identifiers in both DataFrames
df_relevance['product_id_int'] = df_relevance['product_id'].map(id_to_int)
df_ratings['docid_int'] = df_ratings['docid'].map(id_to_int)

In [152]:
df_relevance

,query_id,query_string,product_id,position,relevance,run,product_id_int
0,0,$30 roblox gift card not digital,B00F4CF4PU,0,1.000000,min_maxarithmetic_mean0.0,1
1,0,$30 roblox gift card not digital,B07C438TMN,1,0.474647,min_maxarithmetic_mean0.0,2
2,0,$30 roblox gift card not digital,B00XJZHJCA,2,0.462698,min_maxarithmetic_mean0.0,3
3,0,$30 roblox gift card not digital,B00GAC1D2G,3,0.399549,min_maxarithmetic_mean0.0,4
4,0,$30 roblox gift card not digital,B004RMK4BC,4,0.325092,min_maxarithmetic_mean0.0,5
...,...,...,...,...,...,...,...
145195,219,yarn purple and pink,B079J4V4G8,5,0.311684,l2geometric_mean1.0,2753
145196,219,yarn purple and pink,B07HG28KLX,6,0.309372,l2geometric_mean1.0,2803
145197,219,yarn purple and pink,B08B9K99R9,7,0.277731,l2geometric_mean1.0,3325
145198,219,yarn purple and pink,B01ILAACNU,8,0.276288,l2geometric_mean1.0,3326


In [153]:
df_ratings

,idx,docid,rating,docid_int
0,0,B07RX6FBFR,3,3328
1,0,B09194H44R,0,3329
2,0,B08R5N6W6B,2,3330
3,0,B07Y693ND1,0,3331
4,0,B07RZ75JW3,2,3332
...,...,...,...,...
4060,219,B00JX10Q2O,2,6418
4061,219,B00KY41UHO,3,6419
4062,219,B00QXJOUL2,3,2176
4063,219,B00UY14WCM,3,6420


In [154]:
df_ratings

,idx,docid,rating,docid_int
0,0,B07RX6FBFR,3,3328
1,0,B09194H44R,0,3329
2,0,B08R5N6W6B,2,3330
3,0,B07Y693ND1,0,3331
4,0,B07RZ75JW3,2,3332
...,...,...,...,...
4060,219,B00JX10Q2O,2,6418
4061,219,B00KY41UHO,3,6419
4062,219,B00QXJOUL2,3,2176
4063,219,B00UY14WCM,3,6420


In [163]:
# Drop the docid column as it is not needed
df_pytrec_qrels = df_ratings.drop(columns=['docid'])

df_pytrec_qrels['docid_int'] = df_pytrec_qrels['docid_int'].astype(str)

# Initialize an empty dictionary to store the final qrel structure
qrel = {}

# Group by 'idx' (which corresponds to 'q1', 'q2', etc.)
for idx, group in df_pytrec_qrels.groupby('idx'):
    # Create a dictionary for each group where 'docid' is the key and 'rating' is the value
    qrel[str(idx)] = dict(zip(group['docid_int'], group['rating']))

In [164]:
df_pytrec_results = df_relevance.drop(columns=['position', 'product_id'])

df_pytrec_results['relevance'] = df_pytrec_results['relevance'].astype(int)
df_pytrec_results['product_id_int'] = df_pytrec_results['product_id_int'].astype(str)

# Initialize an empty dictionary to store the final 'run' structure
run = {}

# Group by 'run' and 'query_id' (to ensure we build the correct structure)
for run_value, run_group in df_pytrec_results.groupby('run'):
    # For each 'run' value, initialize an empty dictionary for storing query-specific data
    run[run_value] = {}
    
    # Group by 'query_id' within each 'run' group
    for query_id, query_group in run_group.groupby('query_id'):
        # Create a dictionary where 'product_id' is the key and 'relevance' is the value for each query_id
        run[run_value][query_id] = dict(zip(query_group['product_id_int'], query_group['relevance']))

## Calculate metrics per pipeline and query

In [165]:
evaluator = pytrec_eval.RelevanceEvaluator(
    qrel, {'map', 'ndcg'})

# initialize dict
data = {}

# store search metrics per pipeline
for pipeline, queries in run.items():
    data[pipeline] = evaluator.evaluate(run[pipeline])

## Calculate metrics per pipeline by averaging the results per query

In [166]:
metrics_results = []

for pipeline, queries in data.items():

    ndcg_sum = 0
    map_sum = 0
    num_queries = len(data[pipeline])
    
    # Iterate over the dictionary and sum the 'ndcg' and 'map' values
    for query, metrics in data[pipeline].items():
        ndcg_sum += metrics['ndcg']
        map_sum += metrics['map']
    
    # Calculate the averages
    average_ndcg = ndcg_sum / num_queries
    average_map = map_sum / num_queries
    metrics_results.append({
        'pipeline' : pipeline,
        'avg_ndcg' : average_ndcg,
        'avg_map' : average_map
    })

    # Print the results
    #print(f"Baseline metrics for pipeline {pipeline}")
    #print(f"Average NDCG: {average_ndcg}")
    #print(f"Average MAP: {average_map}")
df_metrics = pd.DataFrame(metrics_results)

In [167]:
df_metrics.sort_values(by='avg_map', ascending=False)

,pipeline,avg_ndcg,avg_map
65,min_maxharmonic_mean1.0,0.278004,0.167524
43,min_maxarithmetic_mean1.0,0.278004,0.167524
54,min_maxgeometric_mean1.0,0.278004,0.167524
6,l2arithmetic_mean0.6,0.261680,0.157890
7,l2arithmetic_mean0.7,0.259861,0.157812
...,...,...,...
1,l2arithmetic_mean0.1,0.195071,0.105125
22,l2harmonic_mean0.0,0.195071,0.105125
11,l2geometric_mean0.0,0.195071,0.105125
2,l2arithmetic_mean0.2,0.195071,0.105125


In [168]:
df_metrics.sort_values(by='avg_ndcg', ascending=False)

,pipeline,avg_ndcg,avg_map
65,min_maxharmonic_mean1.0,0.278004,0.167524
43,min_maxarithmetic_mean1.0,0.278004,0.167524
54,min_maxgeometric_mean1.0,0.278004,0.167524
40,min_maxarithmetic_mean0.7,0.265295,0.157272
41,min_maxarithmetic_mean0.8,0.265004,0.156991
...,...,...,...
1,l2arithmetic_mean0.1,0.195071,0.105125
22,l2harmonic_mean0.0,0.195071,0.105125
11,l2geometric_mean0.0,0.195071,0.105125
2,l2arithmetic_mean0.2,0.195071,0.105125
